In [ ]:
import numpy as np

array = np.load('x_train.npy')

print(array[1])

In [69]:
import numpy as np

def pca_transform(data, n_components):
    # Reshape the data if necessary
    data = data.reshape(data.shape[0], -1)
    
    # Calculate mean
    mean = np.mean(data, axis=0)
    
    # Center the data
    data_centered = data - mean
    
    # Calculate covariance matrix
    cov_matrix = np.cov(data_centered, rowvar=False)
    
    # Compute eigenvectors and eigenvalues
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    
    # Sort eigenvalues and corresponding eigenvectors
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvectors_sorted = eigenvectors[:, sorted_indices]
    
    # Select top n_components eigenvectors
    pca_matrix = eigenvectors_sorted[:, :n_components]
    
    # Project data onto the principal components
    data_reduced = np.dot(data_centered, pca_matrix)
    
    return data_reduced, pca_matrix

# Load data
mnist_data = np.load('mnist.npz')
X_train = mnist_data['x_train']
y_train = mnist_data['y_train']
x_test = mnist_data['x_test']
y_test = mnist_data['y_test']

# Filter out samples from classes 0 and 1
mask_train = (y_train < 2)
X_train_filtered = X_train[mask_train].reshape(-1, 28*28)
y_train_filtered = y_train[mask_train]

mask_test = (y_test < 2)
x_test_filtered = x_test[mask_test]
y_test_filtered = y_test[mask_test]

# Divide the train set into train and val set
X_val = X_train_filtered[:2000]
y_val = y_train_filtered[:2000]
X_train_filtered = X_train_filtered[2000:]
y_train_filtered = y_train_filtered[2000:]

# Apply PCA and reduce the dimensionality to p = 5
X_reduced_train, pca_matrix = pca_transform(X_train_filtered, n_components=5)
X_reduced_val = np.dot(X_val.reshape(X_val.shape[0], -1) - np.mean(X_train_filtered, axis=0), pca_matrix)
x_reduced_test = np.dot(x_test_filtered.reshape(x_test_filtered.shape[0], -1) - np.mean(X_train_filtered, axis=0), pca_matrix)

# Label the classes as -1 and 1
y_train_filtered[y_train_filtered == 0] = -1
y_val[y_val == 0] = -1
y_test_filtered[y_test_filtered == 0] = -1
y_train_filtered[y_train_filtered == 1] = 1
y_val[y_val == 1] = 1
y_test_filtered[y_test_filtered == 1] = 1
print(y_train_filtered.shape)
print(y_val.shape)
print(y_test_filtered.shape)
print("Train set shapes after PCA:", X_reduced_train.shape)
print("Validation set shapes after PCA:", X_reduced_val.shape)
print("Test set shapes after PCA:", x_reduced_test.shape)


(10665,)
(2000,)
(2115,)
Train set shapes after PCA: (10665, 5)
Validation set shapes after PCA: (2000, 5)
Test set shapes after PCA: (2115, 5)


C:\Users\dell\AppData\Local\Temp\ipykernel_13752\1422769840.py:59: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -1 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  y_train_filtered[y_train_filtered == 0] = -1
C:\Users\dell\AppData\Local\Temp\ipykernel_13752\1422769840.py:60: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -1 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  y_val[y_val == 0] = -1
C:\Users\dell\AppData\Local\Temp\ipykernel_13752\1422769840.py:61: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -1 to uint8 will fail in the future.
For th

In [61]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

In [62]:
import numpy as np

class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth 
    
    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)

        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
    
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def gini_index(self, y): 
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):   
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def fit(self, X, Y, sample_weight=None):  
        if sample_weight is None:
            dataset = np.concatenate((X, Y.reshape(-1, 1)), axis=1)
        else:
            dataset = np.concatenate((X, Y.reshape(-1, 1), sample_weight.reshape(-1, 1)), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):  
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    def make_prediction(self, x, tree):
        if tree.value != None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

# Example usage:
# Assuming X_reduced and y_train are your datasets
weights = np.ones(len(y_train[:1000]))  # Initialize weights uniformly
decision_stump = DecisionTreeClassifier(min_samples_split=2, max_depth=1)
decision_stump.fit(X_reduced[:1000], y_train[:1000])
predictions = decision_stump.predict(X_reduced[:1000])

# Calculate error rate
error_rate = np.sum(weights * (predictions != y_train[:1000])) / np.sum(weights)

# Calculate alpha (α1)
alpha = 0.5 * np.log((1 - error_rate) / error_rate)

# Update weights
for i in range(len(weights)):
    if predictions[i] == y_train[i]:
        weights[i] *= np.exp(-alpha)
    else:
        weights[i] *= np.exp(alpha)

# Normalize weights
weights /= np.sum(weights)


In [63]:
# Print the computed alpha (α1)
print("Alpha (α1):", alpha)

# Print the updated weights
print("Updated weights:", weights)


Alpha (α1): 3.1063030478757594
Updated weights: [0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501 0.000501
 0.

In [65]:
import numpy as np

class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth 
    
    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)

        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
    
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def gini_index(self, y): 
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):   
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def fit(self, X, Y, sample_weight=None):  
        if sample_weight is None:
            dataset = np.concatenate((X, Y.reshape(-1, 1)), axis=1)
        else:
            dataset = np.concatenate((X, Y.reshape(-1, 1), sample_weight.reshape(-1, 1)), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):  
        print("X shape",X.shape)
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    def make_prediction(self, x, tree):
        if tree.value != None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)


# Initialize arrays to store alphas and predictions
alphas = []
all_predictions = []
accuracies = []  # To store accuracies on the validation set

# Initialize weights
weights = np.ones(len(y_train[:3000]))  # Initialize weights uniformly

# Iterate to grow 300 decision stumps
for i in range(300):
    # Build decision stump
    decision_stump = DecisionTreeClassifier(min_samples_split=2, max_depth=1)
    decision_stump.fit(X_reduced[:3000], y_train[:3000], sample_weight=weights)  # Use updated weights
    predictions = decision_stump.predict(X_reduced[:3000])

    # Calculate error rate
    error_rate = np.sum(weights * (predictions != y_train[:3000])) / np.sum(weights)

    # Calculate alpha
    alpha = 0.5 * np.log((1 - error_rate) / error_rate)
    alphas.append(alpha)

    # Update weights
    for j in range(len(weights)):
        if predictions[j] == y_train[j]:
            weights[j] *= np.exp(-alpha)
        else:
            weights[j] *= np.exp(alpha)

    # Normalize weights
    weights /= np.sum(weights)
    
    # Store predictions for later evaluation
    all_predictions.append(predictions)
    
    # Compute accuracy on the validation set
    val_predictions = decision_stump.predict(x_reduced_val)
    val_accuracy = np.mean(val_predictions == y_val)
    accuracies.append(val_accuracy)
    
    # Print alpha and h(2) for each iteration
    print(f"Iteration {i+1}: Alpha = {alpha}, h({i+1}) = {decision_stump}, Validation Accuracy = {val_accuracy}")

# Plot accuracy vs. number of trees
plt.plot(range(1, 301), accuracies)
plt.xlabel('Number of Trees')
plt.ylabel('Accuracy on Validation Set')
plt.title('Accuracy vs. Number of Trees')
plt.show()


X shape (3000, 5)
X shape (2000, 5)
Iteration 1: Alpha = 0.06676569631226129, h(1) = <__main__.DecisionTreeClassifier object at 0x000001D7DA521B80>, Validation Accuracy = 0.5345
X shape (3000, 5)


IndexError: index 5 is out of bounds for axis 0 with size 5